In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from glob import glob
# import pandas as ps
import databricks.koalas as ks
ks.set_option('compute.default_index_type', 'distributed')
# Spark session & context
spark = (SparkSession
         .builder
         .master("local")
         .appName("pyspark_pandas_data")
         .config("spark.driver.extraClassPath", "/home/jovyan/work/jars/postgresql-9.4.1207.jar")
         .getOrCreate())

sc = spark.sparkContext

In [5]:
from tqdm import tqdm

concat_arrival_files = '/home/jovyan/work/data/old/concat_arrival_time/arrival_time.csv'
link_folder = '/home/jovyan/work/data/old/test_links/'

d = ks.read_csv(concat_arrival_files)
routes = d.route_num.unique().tolist()
path_list = ['go','back']

for route_num in routes:
    # combine to links
    link=[]
    gps = d[d.route_num==route_num]
    # gps = gps[gps.path == 'go']

    for path in path_list:
        for gps_imei in tqdm(gps.gps_imei.unique().tolist()):
            df = gps[gps.gps_imei == gps_imei]
            df = df[df.path == path]
            # print(f"Current gps imei: {gps_imei}, Current path: {path}")

            for loop in df.loop_num.unique().tolist():
                if len(link)==0:
                    link_name = []
                    link_time = []
                    link_timestamp = []
                    temp = df[df.loop_num == loop]
                    for i in range(len(temp)-1):
                        if temp.station_num.tolist()[i+1] - temp.station_num.tolist()[i] == 1:
                            link_name.append(f'{temp.station_num.tolist()[i]}_{temp.station_num.tolist()[i+1]}')
                            link_time.append((ks.to_datetime(temp.time.iloc[i+1]) - ks.to_datetime(temp.time.iloc[i])).seconds)
                            link_timestamp.append(temp.time.tolist()[i+1])
                    link = ks.DataFrame(link_name)
                    # link.rename(columns={'0':'link_name'})
                    link['link_time(sec)'] = link_time
                    link['link_timestamp'] = link_timestamp
                    link['path'] = [path]*len(link)
                    link['gps_imei'] = [gps_imei]*len(link)
                else:
                    link_name = []
                    link_time = []
                    link_timestamp = []
                    temp = df[df.loop_num == loop]
                    for i in range(len(temp)-1):
                        if temp.station_num.tolist()[i+1] - temp.station_num.tolist()[i] == 1:
                            link_name.append(f'{temp.station_num.tolist()[i]}_{temp.station_num.tolist()[i+1]}')
                            link_time.append((ks.to_datetime(temp.time.iloc[i+1]) - ks.to_datetime(temp.time.iloc[i])).seconds)
                            link_timestamp.append(temp.time.tolist()[i+1])
                    tmp = ks.DataFrame(link_name)
                    # tmp.rename(columns={'0':'link_name'})
                    tmp['link_time(sec)'] = link_time
                    tmp['link_timestamp'] = link_timestamp
                    tmp['path'] = [path]*len(tmp)
                    tmp['gps_imei'] = [gps_imei]*len(tmp)
                    link = ks.concat([link,tmp],axis=0)
    link = link.sort_values('link_timestamp')
    link.columns = ['link_name', 'link_time', 'link_timestamp', 'path', 'gps_imei']
    link['routes'] = [route_num]*len(link)
    # save files
    link.to_csv(link_folder + f'link_{route_num}.csv', index=False)
    break

  0%|          | 0/15 [04:05<?, ?it/s]                              (0 + 1) / 1]


KeyboardInterrupt: 

In [6]:
import pandas as ps

concat_arrival_files = '/home/jovyan/work/data/old/concat_arrival_time/arrival_time.csv'
link_folder = '/home/jovyan/work/data/old/test_links/'

d = ps.read_csv(concat_arrival_files)
routes = d.route_num.unique().tolist()
path_list = ['go','back']

for route_num in routes:
    # combine to links
    link=[]
    gps = d[d.route_num==route_num]
    # gps = gps[gps.path == 'go']

    for path in path_list:
        for gps_imei in tqdm(gps.gps_imei.unique().tolist()):
            df = gps[gps.gps_imei == gps_imei]
            df = df[df.path == path]
            # print(f"Current gps imei: {gps_imei}, Current path: {path}")

            for loop in df.loop_num.unique().tolist():
                if len(link)==0:
                    link_name = []
                    link_time = []
                    link_timestamp = []
                    temp = df[df.loop_num == loop]
                    for i in range(len(temp)-1):
                        if temp.station_num.tolist()[i+1] - temp.station_num.tolist()[i] == 1:
                            link_name.append(f'{temp.station_num.tolist()[i]}_{temp.station_num.tolist()[i+1]}')
                            link_time.append((ps.to_datetime(temp.time.iloc[i+1]) - ps.to_datetime(temp.time.iloc[i])).seconds)
                            link_timestamp.append(temp.time.tolist()[i+1])
                    link = ps.DataFrame(link_name)
                    # link.rename(columns={'0':'link_name'})
                    link['link_time(sec)'] = link_time
                    link['link_timestamp'] = link_timestamp
                    link['path'] = [path]*len(link)
                    link['gps_imei'] = [gps_imei]*len(link)
                else:
                    link_name = []
                    link_time = []
                    link_timestamp = []
                    temp = df[df.loop_num == loop]
                    for i in range(len(temp)-1):
                        if temp.station_num.tolist()[i+1] - temp.station_num.tolist()[i] == 1:
                            link_name.append(f'{temp.station_num.tolist()[i]}_{temp.station_num.tolist()[i+1]}')
                            link_time.append((ps.to_datetime(temp.time.iloc[i+1]) - ps.to_datetime(temp.time.iloc[i])).seconds)
                            link_timestamp.append(temp.time.tolist()[i+1])
                    tmp = ps.DataFrame(link_name)
                    # tmp.rename(columns={'0':'link_name'})
                    tmp['link_time(sec)'] = link_time
                    tmp['link_timestamp'] = link_timestamp
                    tmp['path'] = [path]*len(tmp)
                    tmp['gps_imei'] = [gps_imei]*len(tmp)
                    link = ps.concat([link,tmp],axis=0)
    link = link.sort_values('link_timestamp')
    link.columns = ['link_name', 'link_time', 'link_timestamp', 'path', 'gps_imei']
    link['routes'] = [route_num]*len(link)
    # save files
    link.to_csv(link_folder + f'link_{route_num}.csv', index=False)
    break

 27%|██▋       | 4/15 [00:08<00:25,  2.29s/it]Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.base/java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.base/java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:458)
	at java.base/java.net.ServerSocket.implAccept(ServerSocket.java:565)
	at java.base/java.net.ServerSocket.accept(ServerSocket.java:533)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:64)
100%|██████████| 15/15 [00:31<00:00,  2.08s/it]


In [1]:
import pandas as pd

In [2]:
concat_arrival = pd.read_csv('/home/jovyan/work/data/arrival/m04/concat_arrival_time/arrival_time.csv')

In [3]:
concat_arrival

,station_lat,station_long,distance,time,station_num,loop_num,path,route_num,gps_imei,uuid
0,13.551719,100.787562,18.403002,2022-04-01 04:06:27,0,0,go,133,868998030635219,bf8a8a781a48445999b8145da2aeb853
1,13.555041,100.774513,62.365945,2022-04-01 04:08:42,1,0,go,133,868998030635219,2ae48a5d4fb24e94b9650ee5ddd85490
2,13.555705,100.771890,68.087820,2022-04-01 04:09:27,2,0,go,133,868998030635219,ef39d616a7b54fc79324a65481445c9b
3,13.556467,100.768748,3.402383,2022-04-01 04:09:57,3,0,go,133,868998030635219,e7645e15f8a847419a3e3061ca7500a0
4,13.557094,100.766329,25.830966,2022-04-01 04:10:27,4,0,go,133,868998030635219,e53841fd926d48c8a40a4dffa0668c68
...,...,...,...,...,...,...,...,...,...,...
939955,13.557195,100.766266,80.229484,2022-04-30 22:37:02,97,66,back,133,866381052396907,de47a9205cda4f6ebc1d5d25d9af46ed
939956,13.556606,100.768702,32.622903,2022-04-30 22:37:31,98,66,back,133,866381052396907,f041bbc199f94026a1095083d26b444c
939957,13.555765,100.772015,55.431799,2022-04-30 22:38:16,99,66,back,133,866381052396907,4f9120907ab340c380358a7740b0567f
939958,13.555092,100.774674,13.487088,2022-04-30 22:39:01,100,66,back,133,866381052396907,5b8efe92e8a642d4a2b9c6ff85ff4cb5


In [4]:
concat_arrival.route_num

0         133
1         133
2         133
3         133
4         133
         ... 
939955    133
939956    133
939957    133
939958    133
939959    133
Name: route_num, Length: 939960, dtype: int64